# 🙀 Cat Evaluation

### 📝 Imports

In [114]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader

import albumentations as A

import torchvision
import torchvision.transforms as transforms

import os

from PIL import Image

from cat_discriminator_neural_net import CatDiscriminatorNeuralNet

from tools.augmentation.data_augmenter import DataAugmenter

from cats_dataset import CatsDataset

### 🔧 Config

In [115]:
image_size = 512

saved_model_path = "trained_networks/cat_discriminator.pth"

training_data_path = 'data/test'

bathroom_cat_path = '/bathroom-cat'
captain_dir_path = '/captain'
control_dir_path= '/control'

### 🌐 Create Transforms

In [116]:
transforms = transforms.Compose([
    DataAugmenter(target_image_size=image_size, augment_images=False),
    transforms.ToTensor(),
])

DataAugmenter initialized


### 🤓📝 Load Testing Data

In [117]:



dataset = CatsDataset(
    bathroom_cat_dir=training_data_path + bathroom_cat_path, 
    captain_dir=training_data_path + captain_dir_path, 
    control_dir=training_data_path + control_dir_path, 
    transform=transforms)

dataloader = DataLoader(dataset, shuffle=False, num_workers=4)

### 🥾 Initialize the Neural Net

In [118]:
net = CatDiscriminatorNeuralNet()

if os.path.isfile(saved_model_path):
    net.load_state_dict(torch.load(saved_model_path))

net.cuda()
net.eval()

C:\Users\Dan\AppData\Local\Temp\ipykernel_12000\3838318272.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(saved_model_path))


CatDiscriminatorNeuralNet(
  (conv1): Conv2d(3, 12, kernel_size=(10, 10), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 24, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(24, 48, kernel_size=(5, 5), stride=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv4): Conv2d(48, 96, kernel_size=(3, 3), stride=(1, 1))
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (fc1): Linear(in_features=80736, out_features=1624, bias=True)
  (fc2): Linear(in_features=1624, out_features=864, bias=True)
  (fc3): Linear(in_features=864, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=3, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (dropout2): Dropout(p=0.5, inp

### 🤖 Evaluate

In [ ]:
import numpy


correct = 0
total = 0

def print_predictions(labels, outputs):
    numpy_labels = labels.flatten().cpu().numpy()
    max_label_index = numpy.argmax(numpy_labels)
    actual_label = dataset.index_to_class_name(max_label_index)

    numpy_outputs = outputs.flatten().cpu().numpy()
    max_output_index = numpy.argmax(numpy_outputs)
    predicted_label = dataset.index_to_class_name(max_output_index)


    print("Actual:" + actual_label)
    print("Predicted:" + predicted_label)

    for i in range(len(numpy_outputs)):
        print(f'{dataset.index_to_class_name(i)}: {numpy_outputs[i]:.2f}')

    print("")
    print("")

with torch.no_grad():
    for inputs, labels in dataloader:
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')

        # Forward pass
        outputs = net(inputs)  # outputs: [batch_size, num_classes]

        # Get predicted classes as indices
        _, predicted = torch.max(outputs, dim=1)  

        print_predictions(labels, outputs)
        
        numpy_labels = labels.flatten().cpu().numpy()
        max_label_index = numpy.argmax(numpy_labels)
        actual_label = dataset.index_to_class_name(max_label_index)

        numpy_outputs = outputs.flatten().cpu().numpy()
        max_output_index = numpy.argmax(numpy_outputs)
        predicted_label = dataset.index_to_class_name(max_output_index)

        total += 1
        if predicted_label == actual_label:
            correct += 1

accuracy = (correct / total) * 100
print(f'Accuracy: {accuracy:.2f}%')

Actual:bathroom-cat
Predicted:bathroom-cat
bathroom-cat: 2.99
captain: -0.85
control: -2.40


Actual:bathroom-cat
Predicted:bathroom-cat
bathroom-cat: 1.34
captain: -0.65
control: -0.68


Actual:bathroom-cat
Predicted:bathroom-cat
bathroom-cat: 2.77
captain: 0.29
control: -3.03


Actual:bathroom-cat
Predicted:control
bathroom-cat: -4.32
captain: 1.63
control: 1.78


Actual:bathroom-cat
Predicted:captain
bathroom-cat: -0.05
captain: 0.38
control: -0.38


Actual:bathroom-cat
Predicted:bathroom-cat
bathroom-cat: 2.21
captain: -0.41
control: -1.86


Actual:bathroom-cat
Predicted:bathroom-cat
bathroom-cat: 4.29
captain: -0.97
control: -3.80


Actual:bathroom-cat
Predicted:bathroom-cat
bathroom-cat: 3.41
captain: -0.43
control: -3.24


Actual:bathroom-cat
Predicted:control
bathroom-cat: -1.03
captain: -0.67
control: 1.42


Actual:bathroom-cat
Predicted:bathroom-cat
bathroom-cat: 2.70
captain: -0.63
control: -2.26


Actual:bathroom-cat
Predicted:bathroom-cat
bathroom-cat: 2.15
captain: -0.47


### 💾 Save Progress

In [113]:
torch.save(net.state_dict(), saved_model_path)